<a href="https://colab.research.google.com/github/JDVila/Accelerated_Parallel_Computing_Assignment_06/blob/main/Accelerated_Parallel_Computing_Assignment_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-4lno37vh
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-4lno37vh
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4295 sha256=e1e9e3d545bc30cb94713d91ebcc974ee2107a76499e183271f8bbc855709f86
  Stored in directory: /tmp/pip-ephem-wheel-cache-k58qh6_r/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [3]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [4]:
%%cu
#include <stdio.h>

void helloCPU()
{
  printf("Hello from the CPU.\n");
}

/*
 * The addition of `__global__` signifies that this function
 * should be launced on the GPU.
 */

__global__ void helloGPU()
{
  printf("Hello from the GPU.\n");
}

int main()
{
  int x= 3;
  helloCPU();


  /*
   * Add an execution configuration with the <<<...>>> syntax
   * will launch this function as a kernel on the GPU.
   */

  helloGPU<<<1, 1>>>();

  /*
   * `cudaDeviceSynchronize` will block the CPU stream until
   * all GPU kernels have completed.
   */


  cudaDeviceSynchronize();
}


Hello from the CPU.
Hello from the GPU.



In [5]:
%%cu
#include <stdio.h>

__global__
void saxpy(int n, float a, float *x, float *y)
{
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  printf("GPU active.....Thread %i\n", i);
  if (i < n) y[i] = a * x[i] + y[i];
}

int main(void)
{
  int N = 100;
  float *x, *y, *d_x, *d_y;
  x = (float*)malloc(N*sizeof(float));
  y = (float*)malloc(N*sizeof(float));

  cudaMalloc(&d_x, N*sizeof(float));
  cudaMalloc(&d_y, N*sizeof(float));

  for (int i = 0; i < N; i++) {
    x[i] = i + 0.0f;
    y[i] = i * 2.0f;
  }

  cudaMemcpy(d_x, x, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_y, y, N*sizeof(float), cudaMemcpyHostToDevice);

  // Perform SAXPY on 100 elements
  saxpy<<<(N+255)/256, 256>>>(N, 2.0f, d_x, d_y);

  cudaMemcpy(y, d_y, N*sizeof(float), cudaMemcpyDeviceToHost);

  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = max(maxError, abs(y[i]-4.0f));
  printf("Max error: %f\n", maxError);

  for (int i = 0; i < N; i++) {
    printf("%.2f\n", y[i]);
  }

  cudaFree(d_x);
  cudaFree(d_y);
  free(x);
  free(y);
}

GPU active.....Thread 64
GPU active.....Thread 65
GPU active.....Thread 66
GPU active.....Thread 67
GPU active.....Thread 68
GPU active.....Thread 69
GPU active.....Thread 70
GPU active.....Thread 71
GPU active.....Thread 72
GPU active.....Thread 73
GPU active.....Thread 74
GPU active.....Thread 75
GPU active.....Thread 76
GPU active.....Thread 77
GPU active.....Thread 78
GPU active.....Thread 79
GPU active.....Thread 80
GPU active.....Thread 81
GPU active.....Thread 82
GPU active.....Thread 83
GPU active.....Thread 84
GPU active.....Thread 85
GPU active.....Thread 86
GPU active.....Thread 87
GPU active.....Thread 88
GPU active.....Thread 89
GPU active.....Thread 90
GPU active.....Thread 91
GPU active.....Thread 92
GPU active.....Thread 93
GPU active.....Thread 94
GPU active.....Thread 95
GPU active.....Thread 96
GPU active.....Thread 97
GPU active.....Thread 98
GPU active.....Thread 99
GPU active.....Thread 100
GPU active.....Thread 101
GPU active.....Thread 102
GPU active.....Thread 